In [75]:
import numpy as np
import pandas as pd
import torch
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import urllib.request
import seaborn as sns

np.random.seed(13)

pd.set_option('display.max_columns', None)

# DATA ACQUISITION

In [76]:
datasets_csv = ["movies.csv", "ratings.csv", "genome-scores.csv", "genome-tags.csv", "links.csv", "tags.csv"]
datasets_name = [i[:-4].replace("-", "_") for i in datasets_csv]

root_URL = "http://github.com/MickPerl/DataAnalyticsProject/releases/download/datasets/"
remote_url = ["".join([root_URL, i]) for i in datasets_csv]

In [77]:
try :
    if os.getcwd().split("/")[-1] != 'ml-25m':
        os.chdir('data/ml-25m/')
except FileNotFoundError :
    os.makedirs("data/ml-25m")
    os.chdir('data/ml-25m/')

In [78]:
# per vedere se i csv sono già presenti
!ls

# per assicurarsi di essere nel virtual environment
!which python

genome-scores.csv
genome-tags.csv
links.csv
movies.csv
ratings.csv
tags.csv
/c/Users/miche/Documents/GitHub/DataAnalyticsProject/venv/Scripts/python


In [79]:
for i in range(len(datasets_csv)):
    try :
        globals()["_".join(['df', datasets_name[i]])] = pd.read_csv(datasets_csv[i])
    except FileNotFoundError:
        print(f"Download in progress of {datasets_csv[i]}")
        file = urllib.request.urlretrieve(remote_url[i], datasets_csv[i])
        globals()["_".join(['df', datasets_name[i]])] = pd.read_csv(file[0])

# PRE PROCESSING

In [80]:
df_movies["year"] = [i.strip()[-5:-1] for i in df_movies["title"]]
df_movies["title_length"] = [len(i) for i in df_movies["title"]]
df_movies.drop("title", inplace=True, axis=1)

In [81]:
# SPIEGARE
mlb = MultiLabelBinarizer()
df_movies = df_movies.join(pd.DataFrame(
                mlb.fit_transform(df_movies.pop('genres').str.split('|')),
                index=df_movies.index,
                columns=mlb.classes_))

In [82]:
df_movies['genres'] = df_movies.iloc[:,3:23].values.tolist()
df_movies.head()

,movieId,year,title_length,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres
0,1,1995,16,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2,1995,14,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,3,1995,23,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,1995,24,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,5,1995,34,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Hamming

In [83]:
np.sum(np.bitwise_xor([1,1,0,0,0],[1,0,1,1,0]))

3

In [84]:
df_genome = pd.merge(df_genome_tags, df_genome_scores, on="tagId")
df_genome = df_genome.pivot(index='movieId', columns='tag', values="relevance")
df = pd.merge(df_movies, df_genome, on="movieId")

In [85]:
df_ratings = df_ratings.iloc[:,[1,2]]
df_ratings = df_ratings.groupby(by='movieId').mean()
df = pd.merge(df_ratings, df, on="movieId")

In [86]:
df_X = df.loc[:,df.columns != 'rating']
df_Y = df['rating']

X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=20)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=20)

In [87]:
pd.set_option('display.max_rows', df.shape[0]+1)
df.describe()

movieId        rating  title_length  (no genres listed)  \
count   13816.000000  13816.000000  13816.000000        13816.000000   
mean    46022.487261      3.273803     25.929647            0.003257   
std     55354.212843      0.488670     12.907168            0.056980   
min         1.000000      0.874172      3.000000            0.000000   
25%      3853.750000      2.987988     17.000000            0.000000   
50%      8575.500000      3.341660     22.000000            0.000000   
75%     80186.500000      3.639544     30.000000            0.000000   
max    206499.000000      4.512081    158.000000            1.000000   

             Action     Adventure     Animation      Children        Comedy  \
count  13816.000000  13816.000000  13816.000000  13816.000000  13816.000000   
mean       0.159525      0.109076      0.052258      0.059496      0.347351   
std        0.366178      0.311746      0.222555      0.236560      0.476146   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      1.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

              Crime   Documentary         Drama       Fantasy     Film-Noir  \
count  13816.000000  13816.000000  13816.000000  13816.000000  13816.000000   
mean       0.117834      0.051824      0.477056      0.066445      0.009988   
std        0.322424      0.221680      0.499491      0.249067      0.099445   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      1.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

             Horror          IMAX       Musical       Mystery       Romance  \
count  13816.000000  13816.000000  13816.000000  13816.000000  13816.000000   
mean       0.107846      0.012522      0.035539      0.059062      0.157643   
std        0.310197      0.111202      0.185143      0.235749      0.364419   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

             Sci-Fi      Thriller           War       Western           007  \
count  13816.000000  13816.000000  13816.000000  13816.000000  13816.000000   
mean       0.086783      0.192313      0.040895      0.019687      0.046596   
std        0.281528      0.394132      0.198053      0.138928      0.071478   
min        0.000000      0.000000      0.000000      0.000000      0.007000   
25%        0.000000      0.000000      0.000000      0.000000      0.025750   
50%        0.000000      0.000000      0.000000      0.000000      0.033750   
75%        0.000000      0.000000      0.000000      0.000000      0.045312   
max        1.000000      1.000000      1.000000      1.000000      0.999750   

       007 (series)  18th century         1920s         1930s         1950s  \
count  13816.000000  13816.000000  13816.000000  13816.000000  13816.000000   
mean       0.044771      0.078292      0.085599      0.129250      0.088348   
std        0.062601      0.111595      0.086021      0.146648      0.105167   
min        0.007250      0.006000      0.010000      0.017250      0.004250   
25%        0.026750      0.028750      0.037250      0.053250      0.029500   
50%        0.034750      0.042000      0.0

Select rows with missing data

In [88]:
len(df[pd.isnull(df).any(axis=1)])
# df.fillna(interesting_metric)     or      df.dropna()

0

In [89]:
df.head()

,movieId,rating,year,title_length,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,3d,70mm,80s,9/11,aardman,aardman studios,abortion,absurd,action,action packed,adaptation,adapted from:book,adapted from:comic,adapted from:game,addiction,adolescence,adoption,adultery,adventure,affectionate,afi 100,afi 100 (laughs),afi 100 (movie quotes),africa,afterlife,aging,aids,airplane,airport,alaska,alcatraz,alcoholism,alien,alien invasion,aliens,allegory,almodovar,alone in the world,alter ego,alternate endings,alternate history,alternate reality,alternate universe,amazing cinematography,amazing photography,american civil war,amnesia,amy smart,android(s)/cyborg(s),androids,animal movie,animals,animated,animation,anime,antarctica,anti-hero,anti-semitism,anti-war,apocalypse,archaeology,argentina,arms dealer,arnold,art,art house,artificial intelligence,artist,artistic,artsy,assassin,assassination,assassins,astronauts,atheism,atmospheric,australia,australian,author:alan moore,author:neil gaiman,autism,aviation,awesome,awesome soundtrack,awful,bad,bad acting,bad cgi,bad ending,bad plot,bad science,bad script,bad sequel,ballet,bank robbery,baseball,based on a book,based on a comic,based on a play,based on a true story,based on a tv show,based on a video game,based on book,based on comic,based on true story,basketball,batman,bdsm,beatles,beautiful,beautiful scenery,beautifully filmed,beauty pageant,beer,berlin,best of 2005,best war films,betrayal,better than expected,better than the american version,biblical,big budget,biographical,biography,biopic,birds,biting,bittersweet,bizarre,black and white,black comedy,blaxploitation,bleak,blindness,blood,bloody,boarding school,boat,bollywood,bombs,bond,book,book was better,books,boring,boring!,boston,bowling,boxing,brainwashing,brazil,breathtaking,brilliant,british,british comedy,broadway,brothers,brutal,brutality,buddy movie,bullshit history,bullying,business,c.s. lewis,california,camp,campy,canada,cancer,cannibalism,caper,capitalism,car chase,carrie-anne moss,cars,cartoon,casino,catastrophe,cathartic,catholicism,censorship,cerebral,cgi,character study,chase,cheating,cheerleading,cheesy,chess,chicago,chick flick,child abuse,childhood,children,china,chocolate,chris tucker,christian,christianity,christmas,cia,cinematography,circus,civil war,classic,classic car,classical music,claustrophobic,claymation,clever,clones,cloning,clowns,coen bros,coen brothers,cold war,college,colonialism,colourful,comedy,comic,comic book,comic book adaption,comics,coming of age,coming-of-age,communism,compassionate,competition,complex,complex characters,complicated,complicated plot,computer animation,computer game,computers,con artists,con men,confrontational,confusing,conspiracy,conspiracy theory,controversial,cooking,cool,corny,corporate america,corruption,costume drama,courage,court,courtroom,courtroom drama,crappy sequel,crazy,creativity,creepy,crime,crime gone awry,criterion,cross dressing,crude humor,cuba,cult,cult classic,cult film,culture clash,cute,cute!,cyberpunk,cyborgs,cynical,dance,dancing,dark,dark comedy,dark fantasy,dark hero,dark humor,dc comics,deadpan,death,death penalty,demons,depp & burton,depressing,depression,desert,destiny,detective,devil,dialogue,dialogue driven,dinosaurs,directorial debut,disability,disappointing,disaster,disney,disney animated feature,distopia,disturbing,divorce,doctors,documentary,dog,dogs,dolphins,downbeat,dr. seuss,dragon,dragons,drama,dramatic,dreamlike,dreams,dreamworks,drinking,drug abuse,drug addiction,drugs,dumb,dumb but funny,dynamic cgi action,dysfunctional family,dystopia,dystopic future,earnest,easily confused with other movie(s) (title),east germany,eccentricity,ecology,educational,eerie,effects,egypt,emma watson,emotional,end of the world,england,enigmatic,ennio morricon

DupChecking correlatio

In [73]:
any(df.duplicated())

TypeError: unhashable type: 'list'

## Standardizzazione

Si è valutata fattibilità della standardizzazione sulle relevance dei tags, ma non ha distribuzione gaussiana.

In [50]:
X_train_tags = X_train.iloc[:, 24:]
# X_train_tags_mean = X_train_tags.mean()
# X_train_tags_std = X_train_tags.std()
# X_train.iloc[:, 24:] = (X_train_tags - X_train_tags_mean) / X_train_tags_std
# X_val.iloc[:, 24:] = (X_val.iloc[:, 24:] - X_train_tags_mean) / X_train_tags_std
# X_train.iloc[:, 24:] = (X_test.iloc[:, 24:] - X_train_tags_mean) / X_train_tags_std

In [91]:
X_train_tags.loc[:, "american civil war"].plot(kind ='density')
plt.show()

,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,3d,70mm,80s,9/11,aardman,aardman studios,abortion,absurd,action,action packed,adaptation,adapted from:book,adapted from:comic,adapted from:game,addiction,adolescence,adoption,adultery,adventure,affectionate,afi 100,afi 100 (laughs),afi 100 (movie quotes),africa,afterlife,aging,aids,airplane,airport,alaska,alcatraz,alcoholism,alien,alien invasion,aliens,allegory,almodovar,alone in the world,alter ego,alternate endings,alternate history,alternate reality,alternate universe,amazing cinematography,amazing photography,american civil war,amnesia,amy smart,android(s)/cyborg(s),androids,animal movie,animals,animated,animation,anime,antarctica,anti-hero,anti-semitism,anti-war,apocalypse,archaeology,argentina,arms dealer,arnold,art,art house,artificial intelligence,artist,artistic,artsy,assassin,assassination,assassins,astronauts,atheism,atmospheric,australia,australian,author:alan moore,author:neil gaiman,autism,aviation,awesome,awesome soundtrack,awful,bad,bad acting,bad cgi,bad ending,bad plot,bad science,bad script,bad sequel,ballet,bank robbery,baseball,based on a book,based on a comic,based on a play,based on a true story,based on a tv show,based on a video game,based on book,based on comic,based on true story,basketball,batman,bdsm,beatles,beautiful,beautiful scenery,beautifully filmed,beauty pageant,beer,berlin,best of 2005,best war films,betrayal,better than expected,better than the american version,biblical,big budget,biographical,biography,biopic,birds,biting,bittersweet,bizarre,black and white,black comedy,blaxploitation,bleak,blindness,blood,bloody,boarding school,boat,bollywood,bombs,bond,book,book was better,books,boring,boring!,boston,bowling,boxing,brainwashing,brazil,breathtaking,brilliant,british,british comedy,broadway,brothers,brutal,brutality,buddy movie,bullshit history,bullying,business,c.s. lewis,california,camp,campy,canada,cancer,cannibalism,caper,capitalism,car chase,carrie-anne moss,cars,cartoon,casino,catastrophe,cathartic,catholicism,censorship,cerebral,cgi,character study,chase,cheating,cheerleading,cheesy,chess,chicago,chick flick,child abuse,childhood,children,china,chocolate,chris tucker,christian,christianity,christmas,cia,cinematography,circus,civil war,classic,classic car,classical music,claustrophobic,claymation,clever,clones,cloning,clowns,coen bros,coen brothers,cold war,college,colonialism,colourful,comedy,comic,comic book,comic book adaption,comics,coming of age,coming-of-age,communism,compassionate,competition,complex,complex characters,complicated,complicated plot,computer animation,computer game,computers,con artists,con men,confrontational,confusing,conspiracy,conspiracy theory,controversial,cooking,cool,corny,corporate america,corruption,costume drama,courage,court,courtroom,courtroom drama,crappy sequel,crazy,creativity,creepy,crime,crime gone awry,criterion,cross dressing,crude humor,cuba,cult,cult classic,cult film,culture clash,cute,cute!,cyberpunk,cyborgs,cynical,dance,dancing,dark,dark comedy,dark fantasy,dark hero,dark humor,dc comics,deadpan,death,death penalty,demons,depp & burton,depressing,depression,desert,destiny,detective,devil,dialogue,dialogue driven,dinosaurs,directorial debut,disability,disappointing,disaster,disney,disney animated feature,distopia,disturbing,divorce,doctors,documentary,dog,dogs,dolphins,downbeat,dr. seuss,dragon,dragons,drama,dramatic,dreamlike,dreams,dreamworks,drinking,drug abuse,drug addiction,drugs,dumb,dumb but funny,dynamic cgi action,dysfunctional family,dystopia,dystopic future,earnest,easily confused with other movie(s) (title),east germany,eccentricity,ecology,educational,eerie,effects,egypt,emma watson,emotional,end of the world,england,enigmatic,ennio morricone,enormously long battle scene,ensemble cast,entertaining,entirely dialogue,environment,environmental,epic,erotic,espionage,ethnic conflict,evolution,excellent,excellent script,exceptional acting,exciting,exi

In [ ]:
pd.set_option('expand_frame_repr', False)
list(df.columns)

## Checking correlation

In [ ]:
sns.set_theme(style="ticks")
sns.pairplot(df, hue="species")

In [ ]:
# PRE DATA VISUALIZATION
df_Y.plot(kind ='density')
plt.show()

[] Normalizzazione (valtare se serve stampando distanze tra campioni)\
[] SVM regressione e custom kernel